In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.abspath(os.path.dirname("__file__")), '..'))
from utils import data_handling, model_development
import matplotlib.pyplot as plt
from ipywidgets import interact

In [2]:
##
# Configure the model training policy:
#
model_development.configure_training_policy()

In [3]:
##
# Load and preprocess data:
#
cell_images, cell_labels, cell_types = data_handling.load_data_from_file('../data/labels.csv', True)
print([cell_images.shape, len(cell_labels), len(cell_types)])

@interact(n = (0, cell_images.shape[0] - 1))
def display_data(n = 0):
    fig, axs = plt.subplots()
    axs.imshow(cell_images[n], cmap = 'gray')
    axs.set_title(f'Class: {cell_labels[n]} - Type: {cell_types[n]}')
    return None

[(2624, 300, 300), 2624, 2624]


interactive(children=(IntSlider(value=0, description='n', max=2623), Output()), _dom_classes=('widget-interact…

In [4]:
##
# Prepare data for model training:
#
train_test_split = int(0.85 * cell_images.shape[0])
X_train, Y_train, Z_train  = data_handling.preprocess_data(\
    cell_images[: train_test_split], 
    cell_labels[: train_test_split], 
    cell_types[: train_test_split])
X_test, Y_test, Z_test = \
    cell_images[train_test_split : ], \
    cell_labels[train_test_split : ], cell_types[train_test_split : ]

print([X_train.shape, len(Y_train), len(Z_train)])
print([X_test.shape, len(Y_test), len(Z_test)])

[(2230, 300, 300, 3), 2230, 2230]
[(394, 300, 300), 394, 394]


In [5]:
##
# Train classification model (first phase):
#
optimizer = model_development.create_optimizer('nadam')
resnet152v2 = \
    model_development.resnet152v2(input_shape = (300, 300, 3), weights = 'imagenet', freeze_convolutional_base = True, display_model_information = False)

history, training_time = \
    model_development.train_classification_model(training_phase = 1, model = resnet152v2, 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'ResNet152v2', version = '20221003_fs', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 0.9483 - accuracy: 0.6649 - precision: 0.6763 - recall: 0.5308
Epoch 1: val_accuracy improved from -inf to 0.45373, saving model to ../models/weights\ResNet152v2_20221003_fs.hdf5
119/119 [==============================] - 39s 191ms/step - loss: 0.9483 - accuracy: 0.6649 - precision: 0.6763 - recall: 0.5308 - val_loss: 1.5835 - val_accuracy: 0.4537 - val_precision: 0.4420 - val_recall: 0.9658
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5594 - accuracy: 0.7135 - precision: 0.7489 - recall: 0.5740
Epoch 2: val_accuracy did not improve from 0.45373
119/119 [==============================] - 19s 159ms/step - loss: 0.5594 - accuracy: 0.7135 - precision: 0.7489 - recall: 0.5740 - val_loss: 5.1916 - val_accuracy: 0.4358 - val_precision: 0.4358 - val_recall: 1.0000
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 0.5121 - accuracy: 0.7351 - precision: 0.7857 - recall: 0.5888

In [6]:
##
# Train classification model (second phase):
#
optimizer = model_development.create_optimizer('adam')
history, training_time = \
    model_development.train_classification_model(training_phase = 2, 
    model = '../models/weights/ResNet152v2_20221003_fs.hdf5', 
    optimizer = optimizer, training_metrics = ['accuracy', 'Precision', 'Recall'],
    model_name = 'VGG19', version = '20221003_ss', 
    X = X_train, Y = Y_train, metric_to_monitor = 'val_accuracy', 
    no_of_epochs = 200, batch_size = 16, validation_split_ratio = 0.15)

Epoch 1/200
119/119 [==============================] - ETA: 0s - loss: 12.8425 - accuracy: 0.5821 - precision: 0.5400 - recall: 0.6606
Epoch 1: val_accuracy improved from -inf to 0.68060, saving model to ../models/weights\VGG19_20221003_ss.hdf5
119/119 [==============================] - 54s 353ms/step - loss: 12.8425 - accuracy: 0.5821 - precision: 0.5400 - recall: 0.6606 - val_loss: 0.8415 - val_accuracy: 0.6806 - val_precision: 0.6275 - val_recall: 0.6575
Epoch 2/200
119/119 [==============================] - ETA: 0s - loss: 0.5716 - accuracy: 0.6902 - precision: 0.7928 - recall: 0.4487
Epoch 2: val_accuracy did not improve from 0.68060
119/119 [==============================] - 38s 319ms/step - loss: 0.5716 - accuracy: 0.6902 - precision: 0.7928 - recall: 0.4487 - val_loss: 1.3443 - val_accuracy: 0.5821 - val_precision: 0.5130 - val_recall: 0.8082
Epoch 3/200
119/119 [==============================] - ETA: 0s - loss: 0.5141 - accuracy: 0.7288 - precision: 0.8273 - recall: 0.5239
Epo